In [ ]:
# Perancangan Arsitektur Model

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pyswarms as ps
import joblib

# Mengunci keacakan untuk hasil yang konsisten
np.random.seed(45)
tf.random.set_seed(45)

# Hyperparameter yang akan digunakan
HIDDEN_NODE = 20
N_PARTICLE = 10

# Nama file input (data mentah)
file_input = "Modified_TN_TX_Varied_Gabungan_Semuanya.csv"

try:
    # Memuat dan membersihkan Data Mentah
    df_raw = pd.read_csv(file_input, delimiter=';', na_values=['-', '8888','9999'])
    df_raw.interpolate(method='linear', inplace=True)
    df_cleaned = df_raw.drop('TANGGAL', axis=1)
    print("Berhasil memuat dan membersihkan data mentah.")

    # Mendefinisikan Fitur dan Target
    targets = ['TAVG', 'RH_AVG', 'FF_AVG', 'DDD_X']
    features = ['TN', 'TX', 'RR', 'SS', 'FF_X']
    
    X = df_cleaned[features]
    y = df_cleaned[targets]
    print(f"\nFitur (X) yang digunakan: {features}")
    print(f"Target (y) yang digunakan: {targets}")

    # Normalisasi dan Pembagian Data
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_scaled, test_size=0.2, random_state=45
    )
    print("Data berhasil dinormalisasi dan dibagi (80:20).")

    # Pelatihan Model
    # Fungsi untuk membuat model Backpropagation Neural Network (BPNN)
    def create_bpnn_model(n_inputs, n_outputs):
        model = Sequential() # Membuat model sequential dengan Keras untuk struktur jaringan saraf
        model.add(Dense(HIDDEN_NODE, input_dim=n_inputs, activation='relu')) # Menambahkan lapisan tersembunyi dengan jumlah node HIDDEN_NODE dan aktivasi ReLU
        model.add(Dense(n_outputs, activation='linear')) # Menambahkan lapisan output dengan jumlah node sesuai target (n_outputs) dan aktivasi linear
        return model
    
    # Membuat model sementara untuk optimasi PSO
    temp_model = create_bpnn_model(n_inputs=X_train.shape[1], n_outputs=y_train.shape[1]) # n_inputs diambil dari jumlah kolom X_train, n_outputs dari jumlah kolom y_train

    # Fungsi untuk menetapkan bobot model dari partikel PSO
    def set_model_weights_from_particle(particle, model):
        shapes = [w.shape for w in model.get_weights()]
        args, start = [], 0
        for shape in shapes: # Mengubah partikel menjadi bobot model sesuai dengan bentuk aslinya
            end = start + np.prod(shape)
            args.append(particle[start:end].reshape(shape))
            start = end
        model.set_weights(args) # Menetapkan bobot ke model

    # Fungsi fitness untuk PSO: menghitung Mean Squared Error (MSE) untuk setiap partikel
    def fitness_function(particle, model, X, y):
        set_model_weights_from_particle(particle, model) # Menetapkan bobot dari partikel ke model
        y_pred = model.predict(X, verbose=0)
        mse = np.mean((y - y_pred)**2) # Menghitung MSE antara prediksi dan target aktual
        return mse

    # Fungsi objektif untuk PSO: menghitung MSE untuk seluruh partikel dalam swarm
    def f(swarm_particles, model, X, y):
        n_particles = swarm_particles.shape[0]
        y_np = y.to_numpy() if isinstance(y, pd.DataFrame) else y # Mengonversi y ke numpy array jika dalam bentuk DataFrame
        # Menghitung MSE untuk setiap partikel
        dist = [fitness_function(swarm_particles[i], model, X, y_np) for i in range(n_particles)]
        return np.array(dist)
    
    # Memulai optimasi bobot model menggunakan Particle Swarm Optimization (PSO)
    print(f"\n--- Memulai Optimasi PSO (n_particles={N_PARTICLE}) ---")
    n_params = temp_model.count_params() # Menghitung jumlah parameter (bobot dan bias) dalam model
    options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9} # Menentukan parameter PSO: c1 (kognitif), c2 (sosial), w (inertia)
    # Membuat optimizer PSO dengan jumlah partikel dan dimensi sesuai jumlah parameter
    optimizer = ps.single.GlobalBestPSO(n_particles=N_PARTICLE, dimensions=n_params, options=options)
    # Menjalankan PSO untuk mencari bobot terbaik dengan meminimalkan MSE
    best_cost, best_pos = optimizer.optimize(
        lambda p: f(p, model=temp_model, X=X_train, y=y_train),
        iters=100
    )
    # Menampilkan hasil MSE terendah dari PSO
    print(f"--- Optimasi PSO Selesai. MSE terendah: {best_cost} ---")

    # Memulai pelatihan final model BPNN dengan bobot terbaik dari PSO
    print("\n--- Memulai Pelatihan Final Model BPNN ---")
    final_model = create_bpnn_model(n_inputs=X_train.shape[1], n_outputs=y_train.shape[1])
    # Menetapkan bobot terbaik dari PSO ke model final
    set_model_weights_from_particle(best_pos, final_model)
    # Mengompilasi model dengan optimizer Adam dan loss function MSE
    final_model.compile(optimizer='adam', loss='mean_squared_error')
    # Melatih model dengan data latih, menggunakan data uji untuk validasi
    final_model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
          epochs=800, # Melatih selama 800 epoch
        verbose=1
    )
    # Mengevaluasi performa model pada data uji
    final_loss = final_model.evaluate(X_test, y_test, verbose=0)
    # Menampilkan hasil performa model
    print(f"\n--- HASIL EKSPERIMEN ---")
    print(f"Performa Final Model - MSE pada Data Uji: {final_loss}")

    # Menyimpan Aset Model
    # Nama file diubah untuk mencerminkan fitur baru yang digunakan
    #final_model.save('model_predict_weather_h20_n10_.h5')
    #joblib.dump(scaler_X, 'scaler_X_predict_weather_.pkl')
    #joblib.dump(scaler_y, 'scaler_y_predict_weather_.pkl')
    
    #print("\nModel dan scaler dengan fitur baru berhasil disimpan.")

except FileNotFoundError:
    print(f"Error: File '{file_input}' tidak ditemukan.")
except Exception as e:
    print(f"Terjadi error pada proses utama: {e}")

C:\Users\msi\AppData\Local\Temp\ipykernel_12308\223198998.py:27: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_raw.interpolate(method='linear', inplace=True)
c:\Users\msi\miniforge3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-09-07 21:34:53,989 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


Berhasil memuat dan membersihkan data mentah.

Fitur (X) yang digunakan: ['TN', 'TX', 'RR', 'SS', 'FF_X']
Target (y) yang digunakan: ['TAVG', 'RH_AVG', 'FF_AVG', 'DDD_X']
Data berhasil dinormalisasi dan dibagi (80:20).

--- Memulai Optimasi PSO (n_particles=10) ---


pyswarms.single.global_best: 100%|██████████|100/100, best_cost=0.0692
2025-09-07 21:36:23,276 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.06915458155153553, best pos: [ 0.09568586 -0.30483594 -0.18131632  0.02407837  0.33869829  0.36815996
  0.17602708 -0.70902553 -0.30803385  0.45946519 -0.10479934  0.37888557
 -0.561956   -0.43055167  0.00438768 -0.10364215  0.32318209 -0.01793856
  0.11138768  0.5679315  -0.18588704 -0.55408048 -0.44152372  0.06349274
 -0.17364443 -0.21214267  0.77520872  0.01991855 -0.98782215 -0.38754895
 -0.35428168 -0.30809938 -0.47787107 -0.38361817 -0.030969   -0.49433137
  0.19457763 -0.26258963  0.61642641  0.59511081  0.09357356  0.72305101
 -0.01745457  0.05386827 -0.30434911 -0.06579652  0.89205479 -0.12254806
 -0.40479848 -0.33063489  0.1641124  -0.5314345  -0.07965113 -0.28681368
  0.41967703  0.20481865 -0.06868112 -0.25090882  0.25262644  0.45845195
  0.32297768  0.39929666  0.13411459  0.06577422 -0.19936148  0.033170

--- Optimasi PSO Selesai. MSE terendah: 0.06915458155153553 ---

--- Memulai Pelatihan Final Model BPNN ---
Epoch 1/800
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0700 - val_loss: 0.0631
Epoch 2/800
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0656 - val_loss: 0.0594
Epoch 3/800
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0621 - val_loss: 0.0566
Epoch 4/800
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0593 - val_loss: 0.0542
Epoch 5/800
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0569 - val_loss: 0.0521
Epoch 6/800
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0548 - val_loss: 0.0502
Epoch 7/800
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0530 - val_loss: 0.0484
Epoch 8/800
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0514 - val_loss: 0.0469
Epoch 9/800
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0500 - val_loss: 0.0455
Epoch 10/800
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0488 - val_loss: 0.0443
Epoch 11/800
23/23 ━━━━━━━━━━━━━━━━━━━━ 0

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pyswarms as ps
import joblib
from sklearn.model_selection import ParameterGrid
import uuid

# Mengunci keacakan untuk hasil yang konsisten
np.random.seed(45)
tf.random.set_seed(45)

# Nama file input (data mentah)
file_input = "Modified_TN_TX_Varied_Gabungan_Semuanya.csv"

try:
    # Memuat dan membersihkan Data Mentah
    df_raw = pd.read_csv(file_input, delimiter=';', na_values=['-', '8888','9999'])
    df_raw.interpolate(method='linear', inplace=True)
    df_cleaned = df_raw.drop('TANGGAL', axis=1)
    print("Berhasil memuat dan membersihkan data mentah.")

    # Mendefinisikan Fitur dan Target
    targets = ['TAVG', 'RH_AVG', 'FF_AVG', 'DDD_X']
    features = ['TN', 'TX', 'RR', 'SS', 'FF_X']
    
    X = df_cleaned[features]
    y = df_cleaned[targets]
    print(f"\nFitur (X) yang digunakan: {features}")
    print(f"Target (y) yang digunakan: {targets}")

    # Normalisasi dan Pembagian Data
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_scaled, test_size=0.2, random_state=45
    )
    print("Data berhasil dinormalisasi dan dibagi (80:20).")

    # Fungsi untuk membuat model Backpropagation Neural Network (BPNN)
    def create_bpnn_model(n_inputs, n_outputs, hidden_node):
        model = Sequential()
        model.add(Dense(hidden_node, input_dim=n_inputs, activation='relu'))
        model.add(Dense(n_outputs, activation='linear'))
        return model
    
    # Fungsi untuk menetapkan bobot model dari partikel PSO
    def set_model_weights_from_particle(particle, model):
        shapes = [w.shape for w in model.get_weights()]
        args, start = [], 0
        for shape in shapes:
            end = start + np.prod(shape)
            args.append(particle[start:end].reshape(shape))
            start = end
        model.set_weights(args)

    # Fungsi fitness untuk PSO: menghitung Mean Squared Error (MSE)
    def fitness_function(particle, model, X, y):
        set_model_weights_from_particle(particle, model)
        y_pred = model.predict(X, verbose=0)
        mse = np.mean((y - y_pred)**2)
        return mse

    # Fungsi objektif untuk PSO
    def f(swarm_particles, model, X, y):
        n_particles = swarm_particles.shape[0]
        y_np = y.to_numpy() if isinstance(y, pd.DataFrame) else y
        dist = [fitness_function(swarm_particles[i], model, X, y_np) for i in range(n_particles)]
        return np.array(dist)
    
    # Definisikan parameter grid untuk pencarian
    param_grid = {
        'n_particle': [10, 20],
        'hidden_node': [10, 20],
        'iterations': [50, 75],
        'epochs': [200, 400]
    }

    # Inisialisasi variabel untuk menyimpan hasil terbaik
    best_mse = float('inf')
    best_params = None
    best_model = None
    results = []

    # Lakukan grid search
    print("\n--- Memulai Grid Search untuk Hyperparameter ---")
    for params in ParameterGrid(param_grid):
        print(f"\nMencoba kombinasi: {params}")
        
        # Buat model sementara untuk PSO
        temp_model = create_bpnn_model(
            n_inputs=X_train.shape[1],
            n_outputs=y_train.shape[1],
            hidden_node=params['hidden_node']
        )

        # Optimasi PSO
        n_params = temp_model.count_params()
        options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
        optimizer = ps.single.GlobalBestPSO(
            n_particles=params['n_particle'],
            dimensions=n_params,
            options=options
        )
        best_cost, best_pos = optimizer.optimize(
            lambda p: f(p, model=temp_model, X=X_train, y=y_train),
            iters=params['iterations']
        )
        print(f"PSO selesai. MSE PSO: {best_cost}")

        # Pelatihan final model
        final_model = create_bpnn_model(
            n_inputs=X_train.shape[1],
            n_outputs=y_train.shape[1],
            hidden_node=params['hidden_node']
        )
        set_model_weights_from_particle(best_pos, final_model)
        final_model.compile(optimizer='adam', loss='mean_squared_error')
        final_model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=params['epochs'],
            verbose=0
        )

        # Evaluasi model
        final_loss = final_model.evaluate(X_test, y_test, verbose=0)
        print(f"MSE pada data uji: {final_loss}")

        # Simpan hasil
        results.append({
            'params': params,
            'pso_mse': best_cost,
            'test_mse': final_loss
        })

        # Perbarui model terbaik
        if final_loss < best_mse:
            best_mse = final_loss
            best_params = params
            best_model = final_model

    # Tampilkan hasil grid search
    print("\n--- Hasil Grid Search ---")
    print("Kombinasi hyperparameter terbaik:")
    print(f"Parameter: {best_params}")
    print(f"MSE pada data uji: {best_mse}")

    # Simpan model dan scaler terbaik
    # model_filename = f"model_predict_weather_h{best_params['hidden_node']}_n{best_params['n_particle']}.h5"
    # scaler_X_filename = f"scaler_X_predict_weather_h{best_params['hidden_node']}_n{best_params['n_particle']}.pkl"
    # scaler_y_filename = f"scaler_y_predict_weather_h{best_params['hidden_node']}_n{best_params['n_particle']}.pkl"
    
    best_model.save(model_filename)
    joblib.dump(scaler_X, scaler_X_filename)
    joblib.dump(scaler_y, scaler_y_filename)
    
    print(f"\nModel dan scaler terbaik disimpan sebagai:")
    print(f"Model: {model_filename}")
    print(f"Scaler X: {scaler_X_filename}")
    print(f"Scaler y: {scaler_y_filename}")

except FileNotFoundError:
    print(f"Error: File '{file_input}' tidak ditemukan.")
except Exception as e:
    print(f"Terjadi error pada proses utama: {e}")